In [1]:
pip install --upgrade git+https://github.com/huggingface/diffusers.git transformers accelerate scipy

  Cloning https://github.com/huggingface/diffusers.git to /tmp/pip-req-build-xn_id4ez
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers.git /tmp/pip-req-build-xn_id4ez
  Resolved https://github.com/huggingface/diffusers.git to commit b345c74d4d372e74b87512734b2b64f8f57df6a1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 87.4 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.5/191.5 kB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.8/33.8 MB 39.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 kB 43.6 MB/s eta 0:00:00
  Created wheel for diffusers: filename=diffusers-0.10.2-py3-none-any.whl size=504247 sha256=c1afa299b4db1bbdc70995a707e2781ee0c7f77f575857952a4cba6eac48137f
  Stored in directory: /tmp/pip-ephem-wheel-

In [23]:
import torch
import requests
from PIL import Image
from io import BytesIO

from diffusers import StableDiffusionImg2ImgPipeline, DPMSolverMultistepScheduler

# load the pipeline
device = "cuda"
model_id = "./sd-2-1/stable-diffusion-2-1"
pipe = StableDiffusionImg2ImgPipeline.from_pretrained(model_id, revision="fp16", torch_dtype=torch.float16, use_auth_token=True, safety_checker=None, feature_extractor=None)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)

pipe = pipe.to(device)

init_image = Image.open("./sketch-mountains-input.jpg")
width, height = init_image.size
# init_image = init_image.resize((768, 512))
# import pdb;pdb.set_trace()

prompt = "A fantasy landscape, trending on artstation"
strength = 0.9
guidance_scale = 7.5
num_images_per_prompt = 3
num_inference_steps = 100

generator = torch.Generator(device=device)
seeds = []
for _ in range(num_images_per_prompt):
    seed = generator.seed()
    seeds.append(seed)
    

for i in range(num_images_per_prompt):
    generator = generator.manual_seed(seeds[i])
    PipelineOut = pipe(prompt=prompt, image=init_image, num_inference_steps=num_inference_steps, strength=strength, guidance_scale=guidance_scale, generator=generator)
    images = PipelineOut.images 
    images[0].save("./images/image_{}_{}.png".format(str(i), str(seeds[i])))

  0%|          | 0/90 [00:00<?, ?it/s]

  0%|          | 0/90 [00:00<?, ?it/s]

  0%|          | 0/90 [00:00<?, ?it/s]

In [17]:
seeds

[2888509151495709, 4798389204581365, 7132277553665212]

In [24]:
seed = seeds[2]   
num_images_per_prompt = 1
generator.manual_seed(seed)

PipelineOut = pipe(prompt=prompt, image=init_image, num_inference_steps=num_inference_steps, strength=strength, guidance_scale=guidance_scale, num_images_per_prompt=num_images_per_prompt, generator=generator)
images = PipelineOut.images
images[0].save("./images/final_99.png")

  0%|          | 0/90 [00:00<?, ?it/s]